# Packing stability problem

In [2]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
import math
import numpy as np
import random
import copy

In [3]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['0-out.csv',
 '1-out.csv',
 '2-out.csv',
 '3-out.csv',
 '4-out.csv',
 '5-out.csv',
 '6-out.csv',
 '7-out.csv',
 '8-out.csv',
 '9-out.csv',
 'out_02_01.csv',
 'out_02_02.csv',
 'out_02_03.csv',
 'out_02_04.csv',
 'out_02_05.csv',
 'out_02_06.csv',
 'out_02_07.csv',
 'out_02_08.csv',
 'out_02_09.csv',
 'out_02_10.csv']

In [6]:
def loadData(file_name):
    if file_name == '0-out.csv' or file_name == '5-out.csv' or file_name == '6-out.csv' or file_name == '7-out.csv' or file_name == '8-out.csv' or file_name == '9-out.csv':
        data = read_csv(os.getcwd()+'/'+ file_name, sep=',', index_col=False, skiprows = 1, names=['SKU', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'Aisle', 'Weight'])
    else:
        data = read_csv(os.getcwd()+'/'+ file_name, sep=',', index_col=False, skiprows = 2, names=['SKU', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'Aisle', 'Weight'])
    return data

filename = []
for each in file_list:
    filename.append(each)
datas = []
for each in filename:
    datas.append(loadData(each))
datas[0].head(0)

,SKU,X1,Y1,Z1,X2,Y2,Z2,Aisle,Weight
0,900001,0,0,0,600,400,200,1,12000
1,900001,0,400,0,600,800,200,1,12000
2,900001,600,0,0,1200,400,200,1,12000
3,900001,600,400,0,1200,800,200,1,12000
4,900001,0,0,200,600,400,400,1,12000
5,900001,0,400,200,600,800,400,1,12000
6,900001,600,0,200,1200,400,400,1,12000
7,900001,600,400,200,1200,800,400,1,12000
8,900001,0,0,400,600,400,600,1,12000
9,900001,0,400,400,600,800,600,1,12000


In [7]:
class box():
    def __init__(self, SKU, x_1, y_1, z_1, x_2, y_2, z_2, weight):
        self.SKU = SKU
        self.size = [x_2 - x_1, y_2 - y_1, z_2 - z_1]
        #self.coords = self.size
        #self.rotation = 3
        self.weight = weight
        self.attPoint = [x_1, y_1, z_1]
        self.farCorner = [x_2, y_2, z_2]
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU

    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]

    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]

    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть периметр основания коробки
    def getPerimeter(self):
        return self.size[0] * 2 + self.size[1] * 2

    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)

    #вернуть вес
    def getWeight(self):
        return self.weight

    def __repr__(self):
        return 'Box(SKU=%s, attPoint=%s, farCorner=%s, weight = %s)' % (self.SKU,self.attPoint, self.farCorner, self.weight)

    def getAttPoint(self):
        return list(self.attPoint)

    def getFarCorner(self):
        return list(self.farCorner)

    def getCenterMassX(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0)
    
    def getCenterMassY(self):
        return ((self.farCorner[1]+self.attPoint[1]) / 2.0)
    
    def getCenterMassZ(self):
        return ((self.farCorner[2]+self.attPoint[2]) / 2.0)
        
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))

    def __eq__(self, other):
        if (self.getSKU() == other.getSKU() and self.getFarCorner() == other.getFarCorner() 
                 and self.attPoint() == other.attPoint()):
            return True
        else:
            return False

    def __ne__(self, other):
        if  not (self.farCorner[0] == other.farCorner[0] and 
                 self.farCorner[1] == other.farCorner[1] and 
                 self.farCorner[2] == other.farCorner[2]and 
                 self.attPoint[0] == other.attPoint[0] and 
                 self.attPoint[1] == other.attPoint[1] and 
                 self.attPoint[2] == other.attPoint[2]):
            return True
        else:
            return False

In [8]:
len(datas)

20

In [10]:
palletes = []
for j in range(0, len(datas)):
    boxes = []
    #for i in range(0, len(datas[0])):
    for i in range(0, len(datas[j])):
        #print data['SKU'][i], data['X1'][i], data['Y1'][i], data['Z1'][i], data['X2'][i], data['Y2'][i], data['Z2'][i], data['Weight'][i]
        boxes.append(box(datas[j]['SKU'][i], datas[j]['X1'][i], datas[j]['Y1'][i], datas[j]['Z1'][i], datas[j]['X2'][i], datas[j]['Y2'][i], datas[j]['Z2'][i], datas[j]['Weight'][i]))
    palletes.append(boxes)
palletes[15]

[Box(SKU=790145, attPoint=[0, 0, 0], farCorner=[391, 600, 211], weight = 10727),
 Box(SKU=756584, attPoint=[0, 0, 211], farCorner=[504, 302, 509], weight = 6140),
 Box(SKU=799901, attPoint=[0, 302, 211], farCorner=[508, 608, 409], weight = 8313),
 Box(SKU=747092, attPoint=[391, 0, 0], farCorner=[897, 304, 200], weight = 8260),
 Box(SKU=768211, attPoint=[391, 304, 0], farCorner=[891, 607, 198], weight = 8313),
 Box(SKU=752322, attPoint=[0, 302, 409], farCorner=[423, 612, 617], weight = 9640),
 Box(SKU=760194, attPoint=[0, 0, 509], farCorner=[505, 243, 710], weight = 6900),
 Box(SKU=769016, attPoint=[423, 302, 409], farCorner=[682, 647, 646], weight = 16247),
 Box(SKU=701728, attPoint=[0, 302, 617], farCorner=[207, 501, 1124], weight = 6280),
 Box(SKU=747085, attPoint=[0, 0, 710], farCorner=[506, 202, 909], weight = 8060),
 Box(SKU=779666, attPoint=[207, 302, 617], farCorner=[414, 681, 872], weight = 3680),
 Box(SKU=705053, attPoint=[0, 501, 617], farCorner=[183, 715, 1115], weight = 708

In [11]:
boxess = []
b1 = box(1, 0,0,0,10,10,10, 100)
b2 = box(1, 10,0,0,20,10,10, 100)
boxess.append(b1)
boxess.append(b2)

In [12]:
def getCenterMass(boxes):
    #x_total = boxes[0].getCenterMassX()
    #y_total = boxes[0].getCenterMassY()
    #z_total = boxes[0].getCenterMassZ()
    weight_total = 0
    sum_r_x = 0
    sum_r_y = 0
    sum_r_z = 0
    
    for each in boxes:
        sum_r_x += each.getCenterMassX() * each.getWeight()
        sum_r_y += each.getCenterMassY() * each.getWeight()
        sum_r_z += each.getCenterMassZ() * each.getWeight()
        #x_total = (each.getCenterMassX() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        #y_total = (each.getCenterMassY() * each.getWeight() + y_total * weight_total) / (weight_total + each.getWeight())
        #z_total = (each.getCenterMassZ() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        weight_total += each.getWeight()
    
    cm_x = round(sum_r_x / weight_total, 2)
    cm_y = round(sum_r_y / weight_total, 2)
    cm_z = round(sum_r_z / weight_total, 2)
    
    return (cm_x, cm_y, cm_z)

In [13]:
getCenterMass(palletes[0])

(579.38, 389.69, 403.09)

In [14]:
def perimeter_all(boxes):
    perimeter_all = 0
    for each in boxes:
        perimeter_all += each.getPerimeter()
    return perimeter_all

In [15]:
print perimeter_all(palletes[0])

36000


In [16]:
def perimeter_common(box1, box2):
    
    def getOverlap(a, b):
            return max(0, min(a[1], b[1]) - max(a[0], b[0]))

    def line_x(box1,box2):
        l_x_1 = (box1.attPoint[0], box1.farCorner[0])
        l_x_2 = (box2.attPoint[0], box2.farCorner[0])
        return getOverlap(l_x_1, l_x_2)

    def line_y(box1,box2):
        l_y_1 = (box1.attPoint[1], box1.farCorner[1])
        l_y_2 = (box2.attPoint[1], box2.farCorner[1])
        return getOverlap(l_y_1, l_y_2)
    
    return ((line_x(box1,box2) + line_y(box1,box2)) * 2) if line_x(box1, box2) * line_y(box1, box2) != 0 else 0

In [17]:
def isSameHeight(box1, box2):
    if box1.attPoint[2] == box2.farCorner[2]:
        return True
    else: 
        return False

In [18]:
def perimeter_com_all(boxes):
    per_com_all = 0
    for each in boxes:
        for each1 in boxes:
            if (each != each1 and isSameHeight(each, each1)):
                per_com_all += perimeter_common(each, each1)
    return per_com_all

In [19]:
print perimeter_com_all(palletes[0])

29600


In [20]:
def getPalletHeight(boxes):
    maxheight = 0
    for each in boxes:
        if each.getFarCorner()[2] > maxheight:
            maxheight = each.getFarCorner()[2]
    return maxheight

In [21]:
getPalletHeight(palletes[0])

800

In [22]:
from __future__ import division

In [23]:
def getPerDiffMax(palletes):
    per_diff_max = 0
    for each in palletes:
        if (perimeter_all(each) / (len(each) - 1) - perimeter_com_all(each) / (len(each) - 1) > per_diff_max):
            per_diff_max = (perimeter_all(each) / (len(each) - 1) - perimeter_com_all(each) / (len(each) - 1))
    return per_diff_max

In [24]:
print getPerDiffMax(palletes)

355.775280899


In [25]:
def getCMWeightMax(palletes):
    z_cm_w_max = 0
    for each in palletes:
        if ((getPalletHeight(each) - getCenterMass(each)[2]) / getPalletHeight(each)) > z_cm_w_max:
            z_cm_w_max = ((getPalletHeight(each) - getCenterMass(each)[2]) / getPalletHeight(each))
    return z_cm_w_max

In [26]:
print getCMWeightMax(palletes)

0.543949463482


In [27]:
def stability_coeff(each):
    return ((perimeter_all(each) / (len(each)-1) - perimeter_com_all(each) / (len(each)-1)) / getPerDiffMax(palletes) + 
             ((getPalletHeight(each) - getCenterMass(each)[2]) / getPalletHeight(each)) / getCMWeightMax(palletes)) / 2 

In [29]:
for i in range (0, len(palletes)):
    print 'Stability coefficient of %s pallet: %f' % (filename[i], stability_coeff(palletes[i]))

Stability coefficient of 0-out.csv pallet: 0.884358
Stability coefficient of 1-out.csv pallet: 0.766897
Stability coefficient of 2-out.csv pallet: 0.707443
Stability coefficient of 3-out.csv pallet: 0.831083
Stability coefficient of 4-out.csv pallet: 0.758615
Stability coefficient of 5-out.csv pallet: 0.731260
Stability coefficient of 6-out.csv pallet: 0.708298
Stability coefficient of 7-out.csv pallet: 0.733424
Stability coefficient of 8-out.csv pallet: 0.761998
Stability coefficient of 9-out.csv pallet: 0.676077
Stability coefficient of out_02_01.csv pallet: 0.953818
Stability coefficient of out_02_02.csv pallet: 0.803373
Stability coefficient of out_02_03.csv pallet: 0.911551
Stability coefficient of out_02_04.csv pallet: 0.920694
Stability coefficient of out_02_05.csv pallet: 0.847926
Stability coefficient of out_02_06.csv pallet: 0.869509
Stability coefficient of out_02_07.csv pallet: 0.893642
Stability coefficient of out_02_08.csv pallet: 0.782952
Stability coefficient of out_02_